In [1]:
# 해당 jupyter notebook은 docker를 통한 fenics를 설치한 후 실행해야합니다.
import fenics as fn

mesh = fn.Mesh('geo.xml')
markers = fn.MeshFunction("size_t", mesh, 'geo_physical_region.xml')
boundaries = fn.MeshFunction('size_t', mesh, 'geo_facet_region.xml')
dx = fn.Measure('dx', domain=mesh, subdomain_data=markers)
V = fn.FunctionSpace(mesh, 'Lagrange', 1)

inner_boundary = fn.DirichletBC(V, fn.Constant(5.0), boundaries, 2)
outer_boundary = fn.DirichletBC(V, fn.Constant(0), boundaries, 3)
bcs =[inner_boundary, outer_boundary]

u = fn.TrialFunction(V)
v = fn.TestFunction(V)
a = fn.dot(fn.grad(u), fn.grad(v)) * fn.dx
L = fn.Constant('0') * v * fn.dx
u = fn.Function(V)
fn.solve(a == L, u, bcs)

electric_field = -fn.grad(u)

potentialFile = fn.File('output/potential.pvd')
potentialFile << u

vtkfile = fn.File('output/e_field.pvd')
vtkfile << fn.project(electric_field)
